### 作業目的: 熟練Pytorch Dataset與DataLoader進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [4]:
# Import torch and other required modules
import glob
import torch
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 探索資料與資料前處理
在train資料中，有分成pos(positive)與neg(negative)，分別為正評價與負評價，此評價即為label。

In [6]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf aclImdb_v1.tar.gz

--2021-11-26 17:23:27--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
正在查找主機 ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
正在連接 ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... 連上了。
已送出 HTTP 要求，正在等候回應... 200 OK
長度: 84125825 (80M) [application/x-gzip]
儲存到：「aclImdb_v1.tar.gz」

aclImdb_v1.tar.gz   100%[===================>]  80.23M  2.55MB/s  於 50s       

2021-11-26 17:24:18 (1.61 MB/s) - 已儲存 「aclImdb_v1.tar.gz」 [84125825/84125825]



In [33]:
# 讀取字典，這份字典為review內所有出現的字詞
with open('aclImdb/imdb.vocab', 'r') as f:
    vocab = f.read() 

vocab = vocab.split('\n')

# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
print(f"vocab length before removing stopwords: {len(vocab)}")
vocab = list(set(vocab).difference(set(stopwords.words('english'))))
print(f"vocab length after removing stopwords: {len(vocab)}")

# # 將字典轉換成dictionary
vocab_dic = dict(zip(vocab, range(len(vocab))))
vocab_dic

vocab length before removing stopwords: 89527
vocab length after removing stopwords: 89356


{'baseball-like': 0,
 'uglypeople': 1,
 'demon-possessed': 2,
 'superstars': 3,
 'dealing': 4,
 'cornwall': 5,
 'ice-cold': 6,
 'abducting': 7,
 'navin': 8,
 'sunnygate': 9,
 'breckinridge': 10,
 'beyond-beautiful': 11,
 'aborts': 12,
 'ducky': 13,
 'well-received': 14,
 'bypassed': 15,
 'seductions': 16,
 'inoculated': 17,
 'whalberg': 18,
 'firecrackers': 19,
 'reed': 20,
 'progressed': 21,
 'serenade': 22,
 'pearly': 23,
 'kosher': 24,
 'nut-cases': 25,
 'carefully-constructed': 26,
 'rebelliousness': 27,
 'new-wave': 28,
 'harlem': 29,
 'klaymation': 30,
 'ill-prepared': 31,
 'whiteclad': 32,
 'documentary-not': 33,
 'imagination': 34,
 'hot-cha': 35,
 'snehal': 36,
 'strom': 37,
 'massaging': 38,
 'acosta': 39,
 'syphilis': 40,
 'half-crazy': 41,
 'unfortunates': 42,
 'englishwoman': 43,
 'poetry': 44,
 'milder': 45,
 'budget': 46,
 'twerp': 47,
 'devgn': 48,
 'suknani': 49,
 'steampunk-themed': 50,
 'ipso': 51,
 'tiny': 52,
 'beautifully': 53,
 'afterschool': 54,
 "would'nt": 55,

In [34]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度

review_pos = glob.glob("./aclImdb/train/pos/*.txt")
review_neg = glob.glob("./aclImdb/test/neg/*.txt")
review_all = review_pos + review_neg
y = [1]*len(review_pos) + [0]*len(review_neg)

review_pairs = list(zip(review_all, y))
print(review_pairs[:2])
print(f"Total reviews: {len(review_pairs)}")

[('./aclImdb/train/pos/4715_9.txt', 1), ('./aclImdb/train/pos/12390_8.txt', 1)]
Total reviews: 25000


### 建立Dataset與DataLoader讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量BoW的函式
(generate_bow)

In [35]:
def load_review(review_path):
    with open(review_path, 'r') as f:
        review = f.read()
        
    #移除non-alphabet符號、贅字與tokenize
    review = re.sub('[^a-zA-Z]',' ',review)
    review = nltk.word_tokenize(review)
    review = list(set(review).difference(set(stopwords.words('english'))))
    
    return review

In [36]:
def generate_bow(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
            
    return bag_vector

In [37]:
class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        pair = self.data_dirs[idx]
        review = pair[0]
        review = load_review(review)
        review = generate_bow(review, self.vocab)
        
        return review, pair[1]

In [38]:
# 建立客製化dataset
custom_dst = dataset(review_pairs, vocab_dic)
custom_dst[10]

(array([0., 0., 0., ..., 0., 0., 0.]), 1)

In [41]:
# 建立dataloader
custom_dataloader = DataLoader(dataset=custom_dst, batch_size=4, shuffle=True)
next(iter(custom_dataloader))

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([0, 1, 0, 1])]